In [1]:
# Formula for laminar volumetric flow
%pip install -U sympy

Note: you may need to restart the kernel to use updated packages.


## Importing Packages

In [2]:
import math
import sympy as sp
from sympy.solvers import solve, nsolve
from sympy import Symbol

## Defining Variables

In [3]:
# DEFINE PARAMETERS
l_tube = 0.2  # [m]
l_tank = 0.32  # [m]
w_tank = 0.26  # [m]
h_water_initial = 0.1  # [m]
h_water_final = 0.02  # [m]
d_straight_tube = 7.94 * (10**-3)  # [m]
d_t_joint_tube = 11.25 * (10**-3)  # [m]


# DEFINE RELEVANT SURFACE AREAS
# Area of Tank
area_tank = l_tank * w_tank  # [m^2]
# Area of Straight Tube
area_tube = math.pi * (d_straight_tube / 2) ** 2  # [m^2]
# Area of T-joint Outlets
area_tube_t_joint = 2 * math.pi * (d_t_joint_tube / 2) ** 2  # [m^2]


## DEFINE CONSTANTS
density = 1000  # [kg/m^3]
viscosity = 8.9 * 10**-4  # [Pa*s]
g = 9.81  # [m/s^2]
e = 0.0025 * (10**-3)  # [m]
k = 0.42  # [n/a]

## DEFINE MODELLING VARIABLES
t_interval = 1  # [s], size of time step
t_drain = 0  # [s], total drainage time

## Helper Functions

In [4]:
def reynolds(velocity, length):
    return density * velocity * length / viscosity

## Model Fluid Drainage

In [5]:
# Compute initial velocity
# Assume laminar flow
v_tube = Symbol("v_tube")
v_tube_sol = solve(
    (0.5 * density * ((area_tube / area_tank) ** 2) * (v_tube**2))
    + (density * g * h_water_initial)
    - (0.5 * density * (v_tube**2))
    - (
        (density * g)
        * (64 / ((density * v_tube * d_straight_tube) / viscosity))
        * (l_tube * v_tube**2 / (2 * g * d_straight_tube))
    ),
    v_tube,
)[1]

In [7]:
# Iteratively solve Bernoulli's equation to find total drainage time
h_water = h_water_initial  # [m], set current height of water in tank to initial height

while h_water > h_water_final:
    h_water -= (area_tube / area_tank) * v_tube_sol * t_interval

    # initialize symbols for implicit equation
    v_tube = Symbol("v_tube")
    f = Symbol("f")

    cur_reynolds = reynolds(v_tube_sol, d_straight_tube)
    print(cur_reynolds)

    if cur_reynolds < 2300:
        print("laminar!!")
        friction = 64 / cur_reynolds
    else:
        print("turbulent!!")
        friction = nsolve(
            (1 / sp.sqrt(f))
            + (
                2
                * sp.log(
                    ((e / d_straight_tube) / 3.7) + (2.51 / (cur_reynolds * sp.sqrt(f)))
                )
            ),
            f,
            0.01,
            tol=0.15,
        )
        print(friction)
        # friction = friction[0]

    v_tube_sol = solve(
        (0.5 * density * ((area_tube / area_tank) ** 2) * (v_tube**2))
        + (density * g * h_water)
        - (0.5 * density * (v_tube**2))
        - (
            (density * g)
            * friction
            * (l_tube * (v_tube_sol**2) / (2 * g * d_straight_tube))
        )
        - ((density * g) * (k * ((v_tube_sol**2) / (2 * g)))),
        v_tube,
    )[1]

    print(h_water)

    t_drain += t_interval

11716.1841694937
turbulent!!


TypeError: cannot create mpf from f

In [ ]:
print(t_drain / 60)